# Setup

## Dependencies

In [88]:
import pandas as pd
from datetime import datetime

## Importing Data

In [16]:
# imports the '[W] Customers Table Audit' data
project_data = "./data/customers_table.csv"

# imports '[W] Scope Improvements' data
improvements_data = "./data/improvements_table.csv"

# Parsing Data

## Separate 'In Production' Jobs

**Jobs where '[OB] Completed' and 'Job #' is not blank**

In [14]:
# reading the project data into a df
project_df = pd.read_csv(project_data,
                         dtype={'Claim #': str, 'Job #': str},
                         parse_dates=['Claim # Date', 'FTA Scope. Req Date',
                                      'Submit for Estimate Date', '[OB] Created Scope Calc',
                                      '[B] Created Estimate Date', 'Job Submittal Date',
                                      '[B] - Date Approved by BC', '[OB] Completed'],)

# created 'in_production_df' to hold all current jobs in production, to avoid 'NaN' date values
in_production_df = (project_df.loc[
    (project_df['[OB] Completed'].isnull() == False) & 
    (project_df["Job #"].isnull() == False), :])


## Separate 'Most Recent' Improvement

**Most recent FTA Scope Improvement submitted by the OB**

In [87]:
# created 'improvements_df' to hold all dates of fta scope rejections to use correct date
improvements_df = pd.read_csv(improvements_data,
                              dtype={'Claim #': str, 'Job #': str},
                              parse_dates=['Created'])

# 'improvement_dates_df' holds all current jobs in production, to avoid 'NaN' date values
improvement_dates_df = improvements_df.loc[improvements_df["Job #"].isnull() == False, :]

# 'unique_improv_dates_df' holds most recent 'created' dates for 
unique_improv_dates_df = improvement_dates_df.loc[improvement_dates_df.groupby(
    'Claim #')['Created'].idxmax()]


# will need to update the csv to remove 'submit for estimate' date
unique_improv_dates_df = unique_improv_dates_df.rename(columns = {
    "Claim #" : "Claim #",
    "Job #" : "Job #",
    "Created":"Improvement Date"
})

# Comparing Data

## Merging 'In Production' and 'Most Recent' dataframes

**Merging dfs on the shared 'Claim #' Column**

In [90]:
# merges 'in_production_df' and 'unique_improv_dates_df' on the 'claim #' column
merged_df = pd.merge(in_production_df,
                             unique_improv_dates_df,
                             how= 'left',
                             on="Claim #")


# created the 'workflow_df' as the clean version of the 'merged-df'
workflow_df = pd.DataFrame({
    'Claim #': merged_df['Claim #'],
    'Job #': merged_df['Job #_x'],
    'Claim # Date':merged_df['Claim # Date'],
    'FTA Scope. Req Date': merged_df['FTA Scope. Req Date'],
    'Submit for Estimate Date': merged_df['Submit for Estimate Date'],
    'Improvement Date': merged_df['Improvement Date'],
    '[OB] Created Scope Calc': merged_df['[OB] Created Scope Calc'],
    '[B] Created Estimate Date': merged_df['[B] Created Estimate Date'],
    'Job Submittal Date': merged_df['Job Submittal Date'],
    '[B] - Date Approved by BC': merged_df['[B] - Date Approved by BC'],
    '[OB] Completed': merged_df['[OB] Completed']
})


workflow_df.head()

,Claim #,Job #,Claim # Date,FTA Scope. Req Date,Submit for Estimate Date,Improvement Date,[OB] Created Scope Calc,[B] Created Estimate Date,Job Submittal Date,[B] - Date Approved by BC,[OB] Completed
0,5006023896,1937392,2019-06-24,2019-06-24,2019-06-25,NaT,2019-06-27,2019-06-27,2019-06-27,2019-06-27,2019-06-28 14:52:00
1,001824322-412,1937164,2019-06-17,2019-06-17,2019-06-23,NaT,2019-06-24,2019-06-24,2019-06-27,2019-06-27,2019-06-28 11:05:00
2,017159602-91A-020,1937148,2019-06-15,2019-06-15,2019-06-17,2019-06-18 09:43:00,2019-06-20,2019-06-20,2019-06-27,2019-06-27,2019-06-27 13:59:00
3,040129069,1937101,2019-06-13,2019-06-13,2019-06-14,NaT,2019-06-14,2019-06-17,2019-06-20,2019-06-21,2019-06-21 14:17:00
4,069158K78,1937078,2019-06-13,2019-06-13,2019-06-14,NaT,2019-06-14,2019-06-17,2019-06-27,2019-06-28,2019-06-28 14:54:00


## Creating 'Workflow Days' Value from Merged df

**If 'Improvement Date' exists, use instead of 'Submit for Estimate Date'**

In [91]:
# lists to collect the 'date diffs'
claim_num = []                         
job_num = []
rep_claim_diff = []
fta_scopes_diff = []
ob_scope_diff = []
bc_estimate_diff = []
sup_pfynr_diff = []
bc_approvals_diff = []
ob_order_builds_diff = []
total_days = []

# iterating over the df to create 'date diff' variables
for index, row in workflow_df.iterrows():

    # creating 'date_diff' variables for each step in the workflow
    rep_claim_date_diff = (row["FTA Scope. Req Date"] - row["Claim # Date"]).days
    
    # if 'improvement' date is empty, use 'submit for estimate' for calculations
    if row['Improvement Date'] != row['Improvement Date'] :
        fta_date_diff = (row["Submit for Estimate Date"] - row["FTA Scope. Req Date"]).days
        ob_scope_date_diff = (row['[OB] Created Scope Calc'] - row['Submit for Estimate Date']).days
        fta_scopes_diff.append(fta_date_diff)
        ob_scope_diff.append(ob_scope_date_diff)
#         print(f"{row['Job #']}, Submit for Estimate Date Used")
    
    # ...if not, use 'improvement' date for calculations
    else:
        fta_date_diff = (row["Improvement Date"] - row["FTA Scope. Req Date"]).days
        ob_scope_date_diff = (row['[OB] Created Scope Calc'] - row['Improvement Date']).days
        fta_scopes_diff.append(fta_date_diff)
        ob_scope_diff.append(ob_scope_date_diff)
#         print(f"{row['Job #']}, Improvement Date used")
            
    # creating 'date_diff' variables for each step in the workflow
    bc_estimate_date_diff = (row['[B] Created Estimate Date'] - row['[OB] Created Scope Calc']).days
    sup_pfynr_date_diff = (row["Job Submittal Date"] - row["[B] Created Estimate Date"]).days
    bc_approval_date_diff = (row["[B] - Date Approved by BC"] - row["Job Submittal Date"]).days
    ob_orderbuild_date_diff = (row['[OB] Completed'] - row['[B] - Date Approved by BC']).days
    
    # adding up all of the 'date_diff' variables above and assigning to 'total_days_sum'
    day_diffs = [rep_claim_date_diff, fta_date_diff, ob_scope_date_diff, bc_estimate_date_diff, sup_pfynr_date_diff, bc_approval_date_diff, ob_orderbuild_date_diff]
    total_days_sum = sum(day_diffs)

    # appending 'date diff' values to lists to create each df column
    claim_num.append(row["Claim #"])
    job_num.append(row["Job #"])
    rep_claim_diff.append(rep_claim_date_diff)
    bc_estimate_diff.append(bc_estimate_date_diff)
    sup_pfynr_diff.append(sup_pfynr_date_diff)
    bc_approvals_diff.append(bc_approval_date_diff)
    ob_order_builds_diff.append(ob_orderbuild_date_diff)
    total_days.append(total_days_sum)

# creating the 'days_df' to hold all date values for each role in the project
days_df = pd.DataFrame({
    "claim_#" : claim_num,
    "job_#" : job_num,
    "rep_claim" : rep_claim_diff, 
    "fta_scope" : fta_scopes_diff,
    "ob_scope" : ob_scope_diff,
    "bc_estimate" : bc_estimate_diff,
    "sup_pfynr" : sup_pfynr_diff,
    "bc_approval" : bc_approvals_diff,
    "ob_orderbuild" : ob_order_builds_diff,
    "total_days" : total_days
})

# Export Data

**'In Production' and 'Days' CSVs**

In [92]:
# writes the df to a csv file; 'index=True' writes row names (default)
workflow_df.to_csv("data/in_production.csv", index=False)

days_df.to_csv("data/workflow_days.csv", index=False)